# BERT 임베딩 시각화

## 메타파일 생성

In [ ]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm

def generate_vectors_and_metadata_tsv(path="/content/drive/MyDrive/학교/졸업작품/"):
    input_file = path + "reviews.txt"
    data = []
    with open(input_file, 'r', newline='', encoding='utf-8') as csv_file:
        csv_reader = csv.reader(csv_file)
        for row in csv_reader:
            data.append(row)

    if not data:  # 데이터가 비어있는 경우 처리
        return

    user_history_list = []
    user_ratings_list = []
    user_reviews_list = []
    user_review_emb_list = []

    tmp_reviews = []
    tmp_ratings = []
    tmp_business_id = []
    tmp_review_emb_list = []

    before_user_id = data[0][0]  # 첫 번째 사용자 ID로 초기화
    for idx, i in enumerate(data):
        user_id, business_id, rating, review = i
        if user_id == before_user_id:
            tmp_business_id.append(int(business_id))
            tmp_ratings.append(float(rating))
            tmp_reviews.append(review)
            tmp_review_emb_list.append(idx)
        else:
            if len(tmp_business_id) >= 10:  # 방문 횟수가 2회 이상인 유저만 append
                user_history_list.append(tmp_business_id)
                user_ratings_list.append(tmp_ratings)
                user_reviews_list.append(tmp_reviews)
                user_review_emb_list.append(tmp_review_emb_list)
            tmp_business_id = [int(business_id)]
            tmp_ratings = [float(rating)]
            tmp_reviews = [review]
            tmp_review_emb_list = [idx]
            before_user_id = user_id  # 현재 사용자 ID로 업데이트

    # 마지막 사용자 처리
    if len(tmp_business_id) >= 10:
        user_history_list.append(tmp_business_id)
        user_ratings_list.append(tmp_ratings)
        user_reviews_list.append(tmp_reviews)
        user_review_emb_list.append(tmp_review_emb_list)

    # 메타데이터 생성 및 정리
    metadata = []
    max_user_id = min(5000, len(user_history_list))
    for user_id in range(max_user_id):
        for poi_index in range(len(user_history_list[user_id])):
            poi = user_history_list[user_id][poi_index]
            rating = user_ratings_list[user_id][poi_index]
            review = user_reviews_list[user_id][poi_index].replace('\t', ' ').replace('\n', ' ')
            metadata.append([user_id, poi, rating, review])

    # DataFrame으로 변환
    metadata_df = pd.DataFrame(metadata, columns=["user_id", "poi", "rating", "review"])
    return metadata_df


metadata_df = generate_vectors_and_metadata_tsv()
# 파일로 저장
metadata_df.to_csv("/content/drive/MyDrive/학교/졸업작품/metadata.tsv", sep='\t', index=False)

In [ ]:
metadata_df

,user_id,poi,rating,review
0,0,490,4.0,Aufgrund der günstigen Parkplätze sind wir auc...
1,0,1441,4.0,Ein weiterer Teil des Independence National Hi...
2,0,2438,5.0,In das Macy´s in Philadelphia sind wir nur dur...
3,0,2446,4.0,Eines der meistbesuchten touristischen Ziele i...
4,0,3997,4.0,Wer sich im Independence Visitor Center mit In...
...,...,...,...,...
147839,4999,5367,4.0,Thousand layer bread and coconut curry chicken...
147840,4999,12901,4.0,We enjoyed the gravlax and chicken salad sandw...
147841,4999,12254,5.0,We visited 5 daycares before choosing COA and ...
147842,4999,12254,5.0,One year later and COA continues to be an exce...


In [ ]:
metadata_df.iloc[:10000].to_csv("/content/drive/MyDrive/학교/졸업작품/metadata.tsv", sep='\t', index=False) # 1만개까지 가능

## 벡터 파일 생성

In [ ]:
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import csv

def generate_vectors_tsv(path="/content/drive/MyDrive/학교/졸업작품/"):
    input_file = path + "reviews.txt"
    data = []
    with open(input_file, 'r', newline='', encoding='utf-8') as csv_file:
        csv_reader = csv.reader(csv_file)
        for row in csv_reader:
            data.append(row)

    if not data:  # 데이터가 비어있는 경우 처리
        return

    user_review_emb_list = []

    tmp_review_emb_list = []

    before_user_id = data[0][0]  # 첫 번째 사용자 ID로 초기화
    for idx, i in enumerate(data):
        user_id, business_id, rating, review = i
        if user_id == before_user_id:
            tmp_review_emb_list.append(idx)
        else:
            if len(tmp_review_emb_list) >= 10:  # 방문 횟수가 2회 이상인 유저만 append
                user_review_emb_list.append(tmp_review_emb_list)
            tmp_review_emb_list = [idx]
            before_user_id = user_id  # 현재 사용자 ID로 업데이트

    # 마지막 사용자 처리
    if len(tmp_review_emb_list) >= 10:
        user_review_emb_list.append(tmp_review_emb_list)

    embedding_file = path + 'embeddings.npy'
    embeddings = np.load(embedding_file, mmap_mode='r')

    # DataFrame으로 벡터 저장
    vectors = pd.DataFrame()
    for embeds in tqdm(user_review_emb_list[:5000], desc="Processing Embeddings"):
        if len(embeds) > 0:  # 비어있지 않으면
            new_array = np.array([embeddings[idx] for idx in embeds])
            vectors = pd.concat([vectors, pd.DataFrame(new_array)], ignore_index=True)
        else:
            empty_array = np.zeros((1, 768), dtype=np.float32)
            vectors = pd.concat([vectors, pd.DataFrame(empty_array)], ignore_index=True)

    return vectors

vectors = generate_vectors_tsv()
# vectors.tsv 파일 생성
vectors.to_csv("/content/drive/MyDrive/학교/졸업작품/vectors.tsv", sep='\t', index=False, header=False)

Exception ignored in: <function tqdm.__del__ at 0x78742a0348b0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/tqdm/std.py", line 1148, in __del__
    self.close()
  File "/usr/local/lib/python3.10/dist-packages/tqdm/notebook.py", line 282, in close
    self.disp(bar_style='success', check_delay=False)
AttributeError: 'tqdm' object has no attribute 'disp'


Processing Embeddings:   0%|          | 0/5000 [00:00<?, ?it/s]

In [ ]:
vectors

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,0.120939,-0.038597,-0.046835,-0.049027,0.282406,0.082456,-0.206642,0.019644,0.006271,-0.205330,...,-0.092994,-0.016849,0.086439,-0.138341,0.307182,-0.044859,0.034823,0.068813,-0.077407,0.018856
1,0.027001,-0.007301,-0.058145,-0.031362,0.231266,0.060839,-0.192171,0.042929,-0.047535,-0.244278,...,-0.123042,-0.063056,0.113015,-0.107088,0.297608,-0.053220,0.025348,0.077091,-0.064262,0.087622
2,0.102198,-0.083402,-0.051617,-0.077956,0.331367,0.089070,-0.317232,0.053931,-0.007482,-0.299189,...,-0.131609,-0.041934,0.116763,-0.164791,0.380845,-0.088219,0.055398,0.101921,-0.115275,0.089705
3,0.188088,-0.479429,-0.027945,-0.287908,0.611187,-0.018495,-1.077982,0.257943,0.355617,-0.652930,...,-0.294677,-0.103345,0.227679,-0.190097,0.613229,-0.503943,0.046495,0.135869,-0.178532,0.309895
4,0.049435,0.103430,-0.061820,-0.063628,0.177405,0.076929,0.040645,0.056526,0.018808,-0.133711,...,-0.086080,-0.006858,0.148619,-0.073096,0.194793,-0.022202,-0.011485,0.055195,0.011696,0.027760
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147839,0.371143,-0.982554,0.463354,-0.282904,-0.019903,-0.226173,-1.146576,0.516828,0.374904,-0.128120,...,-0.085816,-0.205268,0.099498,0.637061,0.695560,-0.714824,-0.152146,-0.050416,-0.479860,0.782281
147840,0.036633,-0.634353,-0.015172,-0.247944,0.554902,0.038911,-1.076027,0.210195,0.279697,-0.537716,...,-0.296953,0.009582,0.176845,-0.179773,0.601924,-0.518082,0.060083,0.119459,-0.217469,0.289324
147841,0.290870,-0.926624,0.247915,-0.436004,0.105388,-0.277748,-1.269639,0.551215,0.639693,-0.411574,...,-0.134554,-0.215912,0.215012,0.416519,0.668771,-0.853463,-0.146065,-0.132855,-0.471182,0.755313
147842,-0.022423,0.174037,-0.102672,-0.031502,0.112211,0.063447,0.357431,0.022383,0.031727,0.035515,...,-0.024914,0.081024,0.039877,0.008468,-0.001537,0.082555,-0.008592,0.040650,0.011782,-0.065742


In [ ]:
vectors.iloc[:10000].to_csv("/content/drive/MyDrive/학교/졸업작품/vectors.tsv", sep='\t', index=False, header=False) # 1만개까지 가능